In [ ]:
import arcpy

arcpy.env.workspace = ''
country = 'Rwanda'
in_csv = r'C:\Users\Chris\Downloads\{}_out.csv'.format(country)

points_layer = '{}_points_layer'.format(country)
points_fc = '{}_points_fc'.format(country)
raster_fc = '{}_raster_fc'.format(country)
raster_fc_int = '{}_raster_fc_int'.format(country)
poly_fc = '{}_poly_fc'.format(country)
poly_fc_joined = '{}_poly_fc_joined'.format(country)

arcpy.MakeXYEventLayer_management(in_csv, 'X_deg', 'Y_deg', points_layer, arcpy.SpatialReference('WGS 1984'))
arcpy.CopyFeatures_management (points_layer, points_fc)

arcpy.PointToRaster_conversion(points_fc, 'MinimumOverallCode', raster_fc, 'MAXIMUM', '', '0.009')

outInt = arcpy.sa.Int(raster_fc)
outInt.save(raster_fc_int)

arcpy.RasterToPolygon_conversion(raster_fc_int, poly_fc, 'SIMPLIFY', 'VALUE')

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(poly_fc)
fieldmappings.addTable(points_fc)

field_rules = {'Pop': 'sum', 'NewConnections': 'sum', 'MinimumOverallLCOE': 'median',
               'NewCapacity': 'sum', 'InvestmentCost': 'sum'}

for field, rule in field_rules.items():
    field_index = fieldmappings.findFieldMapIndex(field)
    fieldmap = fieldmappings.getFieldMap(field_index)
    fieldmap.mergeRule = rule
    fieldmappings.replaceFieldMap(field_index, fieldmap)

arcpy.SpatialJoin_analysis(poly_fc, points_fc, poly_fc_joined, '', '', fieldmappings)

delete_fields = ['Join_Count', 'TARGET_FID', 'Id', 'Field1', 'X', 'Y', 'TravelHours',
                'NightLights', 'GridDistCurrent', 'GridDistPlan', 'RoadDist', 'HydropowerDist', 'Hydropower',
                'WindVel', 'GHI', 'SubstationDist', 'LandCover', 'Slope', 'Elevation', 'SolarRestriction',
                'HydropowerFID', 'X_deg', 'Y_deg', 'RoadDistClassified', 'SubstationDistClassified',
                 'LandCoverClassified', 'ElevationClassified', 'SlopeClassified', 'GridClassification',
                 'GridPenalty', 'WindCF', 'PopStartCalibrated', 'IsUrban', 'PopFuture', 'ElecStart',
                'EnergyPerHH', 'NumPeoplePerHH', 'MG_Hydro', 'MG_PV', 'MG_Wind', 'MG_Diesel', 'SA_Diesel',
                 'SA_PV', 'MinimumOffgrid', 'MinimumTechLCOE', 'ElecFuture', 'Grid', 'MinGridDist',
                 'MinimumCategory', 'MinimumOverall', 'MinimumOverallCode']
arcpy.DeleteField_management(poly_fc_joined, delete_fields)
arcpy.AlterField_management (poly_fc_joined, 'gridcode', 'MinimumOverallCode', 'MinimumOverallCode')